In [3]:
import os
from shutil import copy, rmtree
import random

In [4]:
def mk_file(file_path: str):
    if os.path.exists(file_path):
        # 如果文件夹存在，则先删除原文件夹在重新创建
        rmtree(file_path)
    os.makedirs(file_path)

In [19]:
# 保证随机可复现
random.seed(0)

# 将数据集中10%的数据划分到验证集中
split_rate = 0.11

# 指向你解压后的data文件夹
cwd = os.getcwd()
# data_root = os.path.join(cwd, "data_data")#win_powershell
data_root = os.path.abspath(os.path.join(cwd))
origin_data_path = os.path.join(data_root, "train525orig")
assert os.path.exists(origin_data_path), "path '{}' does not exist.".format(origin_data_path)

In [20]:
data_class = [cla for cla in os.listdir(origin_data_path)
                if os.path.isdir(os.path.join(origin_data_path, cla))]

# 建立保存训练集的文件夹
train_root = os.path.join(data_root, "train525s8")
mk_file(train_root)
for cla in data_class:
    # 建立每个类别对应的文件夹
    mk_file(os.path.join(train_root, cla))

# 建立保存验证集的文件夹
val_root = os.path.join(data_root, "val525s8")
mk_file(val_root)
for cla in data_class:
    # 建立每个类别对应的文件夹
    mk_file(os.path.join(val_root, cla))

In [21]:
total_num = 0
for cla in data_class:
    cla_path = os.path.join(origin_data_path, cla)
    images = os.listdir(cla_path)
    num = len(images)
    total_num += num
    # 随机采样验证集的索引
    eval_index = random.sample(images, k=int(num*split_rate))
    for index, image in enumerate(images):
        if image in eval_index:
            # 将分配至验证集中的文件复制到相应目录
            image_path = os.path.join(cla_path, image)
            new_path = os.path.join(val_root, cla)
            copy(image_path, new_path)
        else:
            # 将分配至训练集中的文件复制到相应目录
            image_path = os.path.join(cla_path, image)
            new_path = os.path.join(train_root, cla)
            copy(image_path, new_path)
        print("\r[{}] processing [{}/{}]".format(cla, index+1, num), end="")  # processing bar
    print()

print(f"processing {total_num} done!")

[vasc] processing [132/132]
[nv] processing [6042/6042]
[bkl] processing [1033/1033]
[akiec] processing [304/304]
[mel] processing [1079/1079]
[df] processing [109/109]
[bcc] processing [488/488]
processing 9187 done!


In [22]:
! bash echo_num.sh train525s8 #0.816

==train525s8==
akiec: 	271
bcc: 	435
bkl: 	920
df: 	98
mel: 	961
nv: 	5378
vasc: 	118
total:8181


In [23]:
! bash echo_num.sh val525s8 #0.100

==val525s8==
akiec: 	33
bcc: 	53
bkl: 	113
df: 	11
mel: 	118
nv: 	664
vasc: 	14
total:1006


In [18]:
! ./echo_num.sh test525 #0.082

==test525==
akiec: 	23
bcc: 	26
bkl: 	66
df: 	6
mel: 	34
nv: 	663
vasc: 	10
total:828
